In [224]:
import numpy as np
import pandas as pd
import math
import statsmodels.api as sm
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import LogisticRegression , LinearRegression, SGDClassifier
from sklearn.svm import SVC,SVR
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.svm import SVR
import tensorflow
from sklearn.metrics import r2_score
tensorflow.random.set_seed(1)
from tensorflow.python.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.wrappers.scikit_learn import KerasRegressor


In [117]:
from sklearn.utils import all_estimators

estimators = all_estimators(type_filter='regressor')

all_regs = []
for name, RegressorClass in estimators:
    try:
        print('Appending', name)
        reg = RegressorClass()
        all_regs.append(reg)
    except Exception as e:
        print(e)

Appending ARDRegression
Appending AdaBoostRegressor
Appending BaggingRegressor
Appending BayesianRidge
Appending CCA
Appending DecisionTreeRegressor
Appending DummyRegressor
Appending ElasticNet
Appending ElasticNetCV
Appending ExtraTreeRegressor
Appending ExtraTreesRegressor
Appending GammaRegressor
Appending GaussianProcessRegressor
Appending GradientBoostingRegressor
Appending HistGradientBoostingRegressor
Appending HuberRegressor
Appending IsotonicRegression
Appending KNeighborsRegressor
Appending KernelRidge
Appending Lars
Appending LarsCV
Appending Lasso
Appending LassoCV
Appending LassoLars
Appending LassoLarsCV
Appending LassoLarsIC
Appending LinearRegression
Appending LinearSVR
Appending MLPRegressor
Appending MultiOutputRegressor
__init__() missing 1 required positional argument: 'estimator'
Appending MultiTaskElasticNet
Appending MultiTaskElasticNetCV
Appending MultiTaskLasso
Appending MultiTaskLassoCV
Appending NuSVR
Appending OrthogonalMatchingPursuit
Appending OrthogonalM

In [208]:
pd.set_option('display.max_columns', None)

In [209]:
train_dt = pd.read_csv('train.csv')
train_dt['arrival_date_full'] = train_dt['arrival_date_year'].astype(str) + "-" + train_dt['arrival_date_month'].map({'January':1, 'February':2, 'March':3, 'April':4, 'May':5, 'June':6, 'July':7, 'August':8, 'September':9, 'October':10, 'November':11, 'December':12}).astype(str) + "-" + train_dt['arrival_date_day_of_month'].astype(str)
train_dt['arrival_date_full'] = pd.to_datetime(train_dt['arrival_date_full'], format="%Y-%m-%d")
train_dt['status_minus_arrival_date'] = train_dt['stays_in_week_nights'] + train_dt['stays_in_weekend_nights']
y = train_dt.is_canceled
y_adr = train_dt.adr

In [76]:
train_dt = train_dt.drop(columns=['ID','is_canceled','arrival_date_year',
                                  'arrival_date_month','reservation_status',
                                  'reservation_status_date','adr','agent','company','country'])
train_dt['children']=train_dt['children'].fillna(train_dt['children'].median())


In [77]:
le = LabelEncoder()
categorical_features = list(train_dt.columns[train_dt.dtypes == object])
train_dt[categorical_features]=train_dt[categorical_features].apply(lambda x: le.fit_transform(x))
train_dt.head()

,hotel,lead_time,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,meal,market_segment,distribution_channel,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,reserved_room_type,assigned_room_type,booking_changes,deposit_type,days_in_waiting_list,customer_type,required_car_parking_spaces,total_of_special_requests,arrival_date_full,status_minus_arrival_date
0,1,342,27,1,0,0,2,0.0,0,0,3,1,0,0,0,2,2,3,0,0,2,0,0,2015-07-01,0
1,0,257,27,1,0,2,1,0.0,0,2,5,3,0,0,0,0,0,1,0,0,2,0,0,2015-07-01,2
2,0,257,27,1,0,2,2,0.0,0,2,5,3,0,0,0,0,0,0,0,0,2,0,0,2015-07-01,2
3,0,257,27,1,0,2,2,0.0,0,2,5,3,0,0,0,0,0,0,0,0,2,0,0,2015-07-01,2
4,0,257,27,1,0,2,2,0.0,0,2,5,3,0,0,0,0,0,0,0,0,2,0,0,2015-07-01,2


In [78]:
train_dt.columns

Index(['hotel', 'lead_time', 'arrival_date_week_number',
       'arrival_date_day_of_month', 'stays_in_weekend_nights',
       'stays_in_week_nights', 'adults', 'children', 'babies', 'meal',
       'market_segment', 'distribution_channel', 'is_repeated_guest',
       'previous_cancellations', 'previous_bookings_not_canceled',
       'reserved_room_type', 'assigned_room_type', 'booking_changes',
       'deposit_type', 'days_in_waiting_list', 'customer_type',
       'required_car_parking_spaces', 'total_of_special_requests',
       'arrival_date_full', 'status_minus_arrival_date'],
      dtype='object')

In [79]:
val_import = ['lead_time','previous_bookings_not_canceled','is_repeated_guest'
              ,'booking_changes','required_car_parking_spaces','total_of_special_requests','previous_cancellations']
minmax2 = MinMaxScaler(feature_range=(0,15))
print(minmax2.fit(train_dt[val_import]))
train_dt[val_import] = minmax2.transform(train_dt[val_import])
train_dt

MinMaxScaler(feature_range=(0, 15))


,hotel,lead_time,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,meal,market_segment,distribution_channel,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,reserved_room_type,assigned_room_type,booking_changes,deposit_type,days_in_waiting_list,customer_type,required_car_parking_spaces,total_of_special_requests,arrival_date_full,status_minus_arrival_date
0,1,6.960651,27,1,0,0,2,0.0,0,0,3,1,0.0,0.0,0.0,2,2,2.142857,0,0,2,0.0,0.0,2015-07-01,0
1,0,5.230665,27,1,0,2,1,0.0,0,2,5,3,0.0,0.0,0.0,0,0,0.714286,0,0,2,0.0,0.0,2015-07-01,2
2,0,5.230665,27,1,0,2,2,0.0,0,2,5,3,0.0,0.0,0.0,0,0,0.000000,0,0,2,0.0,0.0,2015-07-01,2
3,0,5.230665,27,1,0,2,2,0.0,0,2,5,3,0.0,0.0,0.0,0,0,0.000000,0,0,2,0.0,0.0,2015-07-01,2
4,0,5.230665,27,1,0,2,2,0.0,0,2,5,3,0.0,0.0,0.0,0,0,0.000000,0,0,2,0.0,0.0,2015-07-01,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91526,1,0.386703,13,31,0,2,2,2.0,0,0,3,1,0.0,0.0,0.0,2,2,0.000000,0,0,2,0.0,3.0,2017-03-31,2
91527,1,0.569878,13,31,0,2,2,0.0,0,0,1,1,0.0,0.0,0.0,0,0,0.000000,0,0,2,0.0,0.0,2017-03-31,2
91528,1,0.040706,13,31,0,1,2,0.0,0,0,6,3,0.0,0.0,0.0,4,5,0.714286,0,0,2,0.0,6.0,2017-03-31,1
91529,1,0.610583,13,31,3,7,2,0.0,0,0,3,1,0.0,0.0,0.0,0,0,0.000000,0,0,2,0.0,3.0,2017-03-31,10


In [80]:
minmax1 = MinMaxScaler()
val_nomal = ['hotel',  'arrival_date_week_number',
       'arrival_date_day_of_month', 'stays_in_weekend_nights',
       'stays_in_week_nights', 'adults', 'children', 'babies', 'meal',
       'market_segment', 'distribution_channel',
       'reserved_room_type', 'assigned_room_type',
       'deposit_type', 'days_in_waiting_list', 'customer_type',
       'status_minus_arrival_date']
print(minmax1.fit(train_dt[val_nomal]))
train_dt[val_nomal] = minmax1.transform(train_dt[val_nomal])
train_dt

MinMaxScaler()


,hotel,lead_time,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,meal,market_segment,distribution_channel,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,reserved_room_type,assigned_room_type,booking_changes,deposit_type,days_in_waiting_list,customer_type,required_car_parking_spaces,total_of_special_requests,arrival_date_full,status_minus_arrival_date
0,1.0,6.960651,0.500000,0.0,0.000000,0.00,0.036364,0.0,0.0,0.0,0.428571,0.25,0.0,0.0,0.0,0.222222,0.181818,2.142857,0.0,0.0,0.666667,0.0,0.0,2015-07-01,0.000000
1,0.0,5.230665,0.500000,0.0,0.000000,0.04,0.018182,0.0,0.0,0.5,0.714286,0.75,0.0,0.0,0.0,0.000000,0.000000,0.714286,0.0,0.0,0.666667,0.0,0.0,2015-07-01,0.028986
2,0.0,5.230665,0.500000,0.0,0.000000,0.04,0.036364,0.0,0.0,0.5,0.714286,0.75,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.666667,0.0,0.0,2015-07-01,0.028986
3,0.0,5.230665,0.500000,0.0,0.000000,0.04,0.036364,0.0,0.0,0.5,0.714286,0.75,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.666667,0.0,0.0,2015-07-01,0.028986
4,0.0,5.230665,0.500000,0.0,0.000000,0.04,0.036364,0.0,0.0,0.5,0.714286,0.75,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.666667,0.0,0.0,2015-07-01,0.028986
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91526,1.0,0.386703,0.230769,1.0,0.000000,0.04,0.036364,0.2,0.0,0.0,0.428571,0.25,0.0,0.0,0.0,0.222222,0.181818,0.000000,0.0,0.0,0.666667,0.0,3.0,2017-03-31,0.028986
91527,1.0,0.569878,0.230769,1.0,0.000000,0.04,0.036364,0.0,0.0,0.0,0.142857,0.25,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.666667,0.0,0.0,2017-03-31,0.028986
91528,1.0,0.040706,0.230769,1.0,0.000000,0.02,0.036364,0.0,0.0,0.0,0.857143,0.75,0.0,0.0,0.0,0.444444,0.454545,0.714286,0.0,0.0,0.666667,0.0,6.0,2017-03-31,0.014493
91529,1.0,0.610583,0.230769,1.0,0.157895,0.14,0.036364,0.0,0.0,0.0,0.428571,0.25,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.666667,0.0,3.0,2017-03-31,0.144928


In [81]:
x = train_dt.drop(columns=['arrival_date_full'])
x

,hotel,lead_time,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,meal,market_segment,distribution_channel,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,reserved_room_type,assigned_room_type,booking_changes,deposit_type,days_in_waiting_list,customer_type,required_car_parking_spaces,total_of_special_requests,status_minus_arrival_date
0,1.0,6.960651,0.500000,0.0,0.000000,0.00,0.036364,0.0,0.0,0.0,0.428571,0.25,0.0,0.0,0.0,0.222222,0.181818,2.142857,0.0,0.0,0.666667,0.0,0.0,0.000000
1,0.0,5.230665,0.500000,0.0,0.000000,0.04,0.018182,0.0,0.0,0.5,0.714286,0.75,0.0,0.0,0.0,0.000000,0.000000,0.714286,0.0,0.0,0.666667,0.0,0.0,0.028986
2,0.0,5.230665,0.500000,0.0,0.000000,0.04,0.036364,0.0,0.0,0.5,0.714286,0.75,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.666667,0.0,0.0,0.028986
3,0.0,5.230665,0.500000,0.0,0.000000,0.04,0.036364,0.0,0.0,0.5,0.714286,0.75,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.666667,0.0,0.0,0.028986
4,0.0,5.230665,0.500000,0.0,0.000000,0.04,0.036364,0.0,0.0,0.5,0.714286,0.75,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.666667,0.0,0.0,0.028986
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91526,1.0,0.386703,0.230769,1.0,0.000000,0.04,0.036364,0.2,0.0,0.0,0.428571,0.25,0.0,0.0,0.0,0.222222,0.181818,0.000000,0.0,0.0,0.666667,0.0,3.0,0.028986
91527,1.0,0.569878,0.230769,1.0,0.000000,0.04,0.036364,0.0,0.0,0.0,0.142857,0.25,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.666667,0.0,0.0,0.028986
91528,1.0,0.040706,0.230769,1.0,0.000000,0.02,0.036364,0.0,0.0,0.0,0.857143,0.75,0.0,0.0,0.0,0.444444,0.454545,0.714286,0.0,0.0,0.666667,0.0,6.0,0.014493
91529,1.0,0.610583,0.230769,1.0,0.157895,0.14,0.036364,0.0,0.0,0.0,0.428571,0.25,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.666667,0.0,3.0,0.144928


In [82]:
x_train, x_val, y_train, y_val = train_test_split(x, y,random_state=45)

In [83]:
x_train.shape

(68648, 24)

In [84]:
y_train

28910    0
83053    0
84282    0
78770    1
50209    0
        ..
12895    0
60960    1
6012     0
63107    0
6558     1
Name: is_canceled, Length: 68648, dtype: int64

In [85]:
rf_params = {
    'n_estimators': [150],  
    'max_features':[5],
    'max_depth': [30]
}

# Perform Grid Search
rf_gs = GridSearchCV(RandomForestClassifier(random_state=42),
                 rf_params,
                 cv = 5,
                 scoring = 'accuracy')
rf = rf_gs.fit(x_train, y_train)
print(f'Best Training Accuracy: {rf.score(x_train, y_train)}')
print(f'Best Testing Accuracy: {rf.score(x_val, y_val)}')

Best Training Accuracy: 0.9852581284232607
Best Testing Accuracy: 0.8782502294279596


In [73]:
test_df = pd.read_csv('test.csv')
test_df['status_minus_arrival_date'] = test_df['stays_in_week_nights'] + test_df['stays_in_weekend_nights']
test_df = test_df.drop(columns=['ID','arrival_date_year',
                                  'arrival_date_month'
                                  ,'agent','company','country'])
categorical_features = list(test_df.columns[test_df.dtypes == object])
test_df[categorical_features]=test_df[categorical_features].apply(lambda x: le.fit_transform(x))

val_import = ['lead_time','previous_bookings_not_canceled','is_repeated_guest'
              ,'booking_changes','required_car_parking_spaces','total_of_special_requests','previous_cancellations']
#minmax2 = MinMaxScaler(feature_range=(0,5))
#print(minmax2.fit(train_dt[val_import]))
test_df[val_import] = minmax2.transform(test_df[val_import])
val_nomal = ['hotel',  'arrival_date_week_number',
       'arrival_date_day_of_month', 'stays_in_weekend_nights',
       'stays_in_week_nights', 'adults', 'children', 'babies', 'meal',
       'market_segment', 'distribution_channel',
       'reserved_room_type', 'assigned_room_type',
       'deposit_type', 'days_in_waiting_list', 'customer_type',
       'status_minus_arrival_date']
#print(minmax1.fit(train_dt[val_nomal]))
test_df[val_nomal] = minmax1.transform(test_df[val_nomal])
test_df




,hotel,lead_time,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,meal,market_segment,distribution_channel,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,reserved_room_type,assigned_room_type,booking_changes,deposit_type,days_in_waiting_list,customer_type,required_car_parking_spaces,total_of_special_requests,status_minus_arrival_date
0,0.0,0.508820,0.230769,0.0,0.105263,0.10,0.036364,0.0,0.0,0.00,0.857143,0.75,0.0,0.0,0.000000,0.333333,0.272727,0.000000,0.0,0.0,0.666667,0.000,1.0,0.101449
1,0.0,1.411126,0.230769,0.0,0.210526,0.20,0.036364,0.0,0.0,0.75,0.857143,0.75,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.666667,0.000,1.0,0.202899
2,1.0,0.081411,0.230769,0.0,0.105263,0.10,0.036364,0.0,0.0,1.00,0.714286,0.75,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000,1.0,0.101449
3,0.0,0.515604,0.230769,0.0,0.105263,0.10,0.054545,0.0,0.0,0.00,0.857143,0.75,0.0,0.0,0.000000,0.333333,0.272727,0.000000,0.0,0.0,0.666667,0.000,0.0,0.101449
4,0.0,0.061058,0.230769,0.0,0.105263,0.08,0.036364,0.2,0.0,0.00,0.857143,0.75,0.0,0.0,0.000000,0.555556,0.454545,0.000000,0.0,0.0,0.666667,0.000,0.0,0.086957
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27854,1.0,0.732700,0.653846,1.0,0.105263,0.10,0.036364,0.0,0.0,0.50,0.857143,0.75,0.0,0.0,0.000000,0.444444,0.363636,0.000000,0.0,0.0,0.666667,0.000,1.0,0.101449
27855,1.0,1.316147,0.653846,1.0,0.105263,0.10,0.036364,0.1,0.0,0.50,0.857143,0.75,0.0,0.0,0.000000,0.666667,0.545455,0.714286,0.0,0.0,0.666667,0.625,1.0,0.101449
27856,1.0,0.115332,0.653846,1.0,0.000000,0.06,0.036364,0.0,0.0,0.50,0.857143,0.75,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.666667,0.000,2.0,0.043478
27857,1.0,1.295794,0.653846,1.0,0.105263,0.10,0.036364,0.0,0.0,0.50,0.714286,0.75,0.0,0.0,0.000000,0.333333,0.272727,0.000000,0.0,0.0,0.000000,0.000,0.0,0.101449


In [74]:
can_pred = rf.predict(test_df)
can_pred

array([0, 1, 0, ..., 0, 0, 0])

In [43]:
rf_params2 = [[150,300],  
            [5,10,20],
            [10,20,30]]
rf_score = []
rf_gs = RandomForestClassifier()
rf_clf = rf_gs.fit(x_train,y_train)
temp = [rf_clf.score(x_train,y_train),rf_clf.score(x_val,y_val)]
rf_score.append(temp)
print(f'Best Training Accuracy: {rf_clf.score(x_train, y_train)}')
print(f'Best Testing Accuracy: {rf_clf.score(x_val, y_val)}')
for i in range(2):
    rf_gs = RandomForestClassifier(n_estimators=rf_params2[0][i])
    rf_clf = rf_gs.fit(x_train,y_train)
    temp = [rf_clf.score(x_train,y_train),rf_clf.score(x_val,y_val)]
    rf_score.append(temp)
    print('i: ',rf_params2[0][i])
    print(f'Best Training Accuracy: {rf_clf.score(x_train, y_train)}')
    print(f'Best Testing Accuracy: {rf_clf.score(x_val, y_val)}')
    for j in range(3):
        rf_gs = RandomForestClassifier(n_estimators=rf_params2[0][i],max_features=rf_params2[1][j])
        rf_clf = rf_gs.fit(x_train,y_train)
        temp = [rf_clf.score(x_train,y_train),rf_clf.score(x_val,y_val)]
        rf_score.append(temp)
        print('i: ',rf_params2[0][i], 'j: ',rf_params2[1][j])
        print(f'Best Training Accuracy: {rf_clf.score(x_train, y_train)}')
        print(f'Best Testing Accuracy: {rf_clf.score(x_val, y_val)}')
        for k in range(3):
            rf_gs = RandomForestClassifier(n_estimators=rf_params2[0][i],max_features=rf_params2[1][j],max_depth=rf_params2[2][k])
            rf_clf = rf_gs.fit(x_train,y_train)
            temp = [rf_clf.score(x_train,y_train),rf_clf.score(x_val,y_val)]
            rf_score.append(temp)
            print('i: ',rf_params2[0][i], 'j: ',rf_params2[1][j], 'k: ',rf_params2[2][k])
            print(f'Best Training Accuracy: {rf_clf.score(x_train, y_train)}')
            print(f'Best Testing Accuracy: {rf_clf.score(x_val, y_val)}')

Best Training Accuracy: 0.9885939867148351
Best Testing Accuracy: 0.8761089018048333
i:  150
Best Training Accuracy: 0.9885939867148351
Best Testing Accuracy: 0.876545907442206
i:  150 j:  5
Best Training Accuracy: 0.9885939867148351
Best Testing Accuracy: 0.877026613643316
i:  150 j:  5 k:  10
Best Training Accuracy: 0.8422678009555996
Best Testing Accuracy: 0.8363413888039156
i:  150 j:  5 k:  20
Best Training Accuracy: 0.9163267684419065
Best Testing Accuracy: 0.8688983087881834
i:  150 j:  5 k:  30
Best Training Accuracy: 0.985534902691994
Best Testing Accuracy: 0.8782939299916969
i:  150 j:  10
Best Training Accuracy: 0.9885939867148351
Best Testing Accuracy: 0.877332517589477
i:  150 j:  10 k:  10
Best Training Accuracy: 0.8407965272112807
Best Testing Accuracy: 0.8349429707643229
i:  150 j:  10 k:  20
Best Training Accuracy: 0.9332828341685119
Best Testing Accuracy: 0.8734868679805969
i:  150 j:  10 k:  30
Best Training Accuracy: 0.9880841393776949
Best Testing Accuracy: 0.87724

In [44]:
dt_params2 = [[5,10,15],[5,10,20],[15,25],[1,3,5]]

dt_score = []
dt_gs = DecisionTreeClassifier()
dt_clf = dt_gs.fit(x_train,y_train)
temp = [dt_clf.score(x_train,y_train),dt_clf.score(x_val,y_val)]
dt_score.append(temp)
print(f'Best Training Accuracy: {dt_clf.score(x_train, y_train)}')
print(f'Best Testing Accuracy: {dt_clf.score(x_val, y_val)}')
for i in range(3):
    dt_gs = DecisionTreeClassifier(max_depth=dt_params2[0][i])
    dt_clf = dt_gs.fit(x_train,y_train)
    temp = [dt_clf.score(x_train,y_train),dt_clf.score(x_val,y_val)]
    dt_score.append(temp)
    print('i: ',dt_params2[0][i])
    print(f'Best Training Accuracy: {dt_clf.score(x_train, y_train)}')
    print(f'Best Testing Accuracy: {dt_clf.score(x_val, y_val)}')
    for j in range(3):
        dt_gs = DecisionTreeClassifier(max_depth=dt_params2[0][i],max_features=dt_params2[1][j])
        dt_clf = dt_gs.fit(x_train,y_train)
        temp = [dt_clf.score(x_train,y_train),dt_clf.score(x_val,y_val)]
        dt_score.append(temp)
        print('i: ',dt_params2[0][i], 'j: ',dt_params2[1][j])
        print(f'Best Training Accuracy: {dt_clf.score(x_train, y_train)}')
        print(f'Best Testing Accuracy: {dt_clf.score(x_val, y_val)}')
        for k in range(2):
            dt_gs = DecisionTreeClassifier(max_depth=dt_params2[0][i],max_features=dt_params2[1][j],min_samples_split=dt_params2[2][k])
            dt_clf = dt_gs.fit(x_train,y_train)
            temp = [dt_clf.score(x_train,y_train),dt_clf.score(x_val,y_val)]
            dt_score.append(temp)
            print('i: ',dt_params2[0][i], 'j: ',dt_params2[1][j], 'k: ',dt_params2[2][k])
            print(f'Best Training Accuracy: {dt_clf.score(x_train, y_train)}')
            print(f'Best Testing Accuracy: {dt_clf.score(x_val, y_val)}')
            for l in range(3):
                dt_gs = DecisionTreeClassifier(max_depth=dt_params2[0][i],max_features=dt_params2[1][j],min_samples_split=dt_params2[2][k],min_samples_leaf=dt_params2[3][l])
                dt_clf = dt_gs.fit(x_train,y_train)
                temp = [dt_clf.score(x_train,y_train),dt_clf.score(x_val,y_val)]
                dt_score.append(temp)
                print('i: ',dt_params2[0][i], 'j: ',dt_params2[1][j], 'k: ',dt_params2[2][k],'l: ',dt_params2[3][l])
                print(f'Best Training Accuracy: {dt_clf.score(x_train, y_train)}')
                print(f'Best Testing Accuracy: {dt_clf.score(x_val, y_val)}')

Best Training Accuracy: 0.9885939867148351
Best Testing Accuracy: 0.8362102871127037
i:  5
Best Training Accuracy: 0.8242629064211631
Best Testing Accuracy: 0.8187300616177948
i:  5 j:  5
Best Training Accuracy: 0.7572252651206153
Best Testing Accuracy: 0.7552768430712756
i:  5 j:  5 k:  15
Best Training Accuracy: 0.7919385852464748
Best Testing Accuracy: 0.7885329720753398
i:  5 j:  5 k:  15 l:  1
Best Training Accuracy: 0.7928271763197763
Best Testing Accuracy: 0.7899313901149325
i:  5 j:  5 k:  15 l:  3
Best Training Accuracy: 0.821844773336441
Best Testing Accuracy: 0.8172442424507276
i:  5 j:  5 k:  15 l:  5
Best Training Accuracy: 0.7642320242395991
Best Testing Accuracy: 0.7624874360879256
i:  5 j:  5 k:  25
Best Training Accuracy: 0.769228528143573
Best Testing Accuracy: 0.7666826902067037
i:  5 j:  5 k:  25 l:  1
Best Training Accuracy: 0.7923755972497378
Best Testing Accuracy: 0.7895817856050343
i:  5 j:  5 k:  25 l:  3
Best Training Accuracy: 0.7809404498310221
Best Testing 

i:  15 j:  20 k:  15
Best Training Accuracy: 0.8641621023190771
Best Testing Accuracy: 0.8411921513787528
i:  15 j:  20 k:  15 l:  1
Best Training Accuracy: 0.8651526628598065
Best Testing Accuracy: 0.8432460778744045
i:  15 j:  20 k:  15 l:  3
Best Training Accuracy: 0.8635502855145089
Best Testing Accuracy: 0.8410173491238037
i:  15 j:  20 k:  15 l:  5
Best Training Accuracy: 0.8626616944412073
Best Testing Accuracy: 0.8434208801293537
i:  15 j:  20 k:  25
Best Training Accuracy: 0.8618896399021093
Best Testing Accuracy: 0.8429401739282437
i:  15 j:  20 k:  25 l:  1
Best Training Accuracy: 0.863521151380958
Best Testing Accuracy: 0.8442074902766246
i:  15 j:  20 k:  25 l:  3
Best Training Accuracy: 0.8611030182962359
Best Testing Accuracy: 0.8449066992964209
i:  15 j:  20 k:  25 l:  5
Best Training Accuracy: 0.8609573476284815
Best Testing Accuracy: 0.8423283660359219


In [ ]:
dt_params = {
    'max_depth':[5,7,9],
    'max_features' : [5,10,20],
    'min_samples_split': [15,25], 
    'min_samples_leaf': [1]
}

# Perform Grid Search
dt_gs = GridSearchCV(DecisionTreeClassifier(random_state=42),
                 dt_params,
                 cv = 5,
                 scoring = 'accuracy')
dt = dt_gs.fit(x_train, y_train)
print(f'Best Training Accuracy: {dt.score(x_train, y_train)}')
print(f'Best Testing Accuracy: {dt.score(x_val, y_val)}')

Best Training Accuracy: 0.0
Best Testing Accuracy: -0.00010099351112069499


In [50]:
lg_params = {
    'penalty': ['l1','l2', 'elasticnet', 'none'], 
    'C': [0.1,1,3],
    'max_iter': [150,300]
}

# Perform Grid Search
lg_gs = GridSearchCV(LogisticRegression(solver='liblinear', random_state=42),
                 lg_params,
                 cv = 5,
                 scoring = 'accuracy')
lg = lg_gs.fit(x_train, y_train)

print(f'Best Training Accuracy: {lg.score(x_train, y_train)}')
print(f'Best Testing Accuracy: {lg.score(x_val, y_val)}')

/usr/local/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:552: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/site-packages/sklearn/model_selection/_validation.py", line 531, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py", line 1304, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "/usr/local/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py", line 450, in _check_solver
    " got solver={}.".format(solver))
ValueError: Only 'saga' solver supports elasticnet penalty, got solver=liblinear.

  FitFailedWarning)
/usr/local/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:552: FitFailedWarning: Estimator fit failed. The score on this train-test partition for the

/usr/local/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:552: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/site-packages/sklearn/model_selection/_validation.py", line 531, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py", line 1304, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "/usr/local/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py", line 450, in _check_solver
    " got solver={}.".format(solver))
ValueError: Only 'saga' solver supports elasticnet penalty, got solver=liblinear.

  FitFailedWarning)
/usr/local/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:552: FitFailedWarning: Estimator fit failed. The score on this train-test partition for the

Best Training Accuracy: 0.8120120032630229
Best Testing Accuracy: 0.8072368133548923


In [57]:
sgd_params = {
    'penalty': ['l2', 'elasticnet'], 
    'alpha': [000.1,0.001,0.01],
}

# Perform Grid Search
sgd_gs = GridSearchCV(SGDClassifier(),
                 sgd_params,
                 cv = 5,
                 scoring = 'accuracy')
sgd = sgd_gs.fit(x_train, y_train)

print(f'Best Training Accuracy: {sgd.score(x_train, y_train)}')
print(f'Best Testing Accuracy: {sgd.score(x_val, y_val)}')

Best Training Accuracy: 0.768456473604475
Best Testing Accuracy: 0.7659834811869073


In [62]:
svc_params = {
    'C': [1,10,100], 
    'kernal': ['rbf','linear','poly'],
}

# Perform Grid Search
svc_gs = GridSearchCV(SVC(),
                 svc_params,
                 cv = 5,
                 scoring = 'accuracy')
svc = svc_gs.fit(x_train, y_train)

print(f'Best Training Accuracy: {svc.score(x_train, y_train)}')
print(f'Best Testing Accuracy: {svc.score(x_val, y_val)}')

ValueError: Invalid parameter kernal for estimator SVC(C=1). Check the list of available parameters with `estimator.get_params().keys()`.

In [65]:
svc = SVC(C=5)
svc.fit(x_train, y_train)

print(f'Best Training Accuracy: {svc.score(x_train, y_train)}')
print(f'Best Testing Accuracy: {svc.score(x_val, y_val)}')

Best Training Accuracy: 0.8338480363593986
Best Testing Accuracy: 0.8272953721103002


# +++    ++++++

In [211]:
train_dt = pd.read_csv('train.csv')
# train_dt['arrival_date_full'] = train_dt['arrival_date_year'].astype(str) + "-" + train_dt['arrival_date_month'].map({'January':1, 'February':2, 'March':3, 'April':4, 'May':5, 'June':6, 'July':7, 'August':8, 'September':9, 'October':10, 'November':11, 'December':12}).astype(str) + "-" + train_dt['arrival_date_day_of_month'].astype(str)
# train_dt['arrival_date_full'] = pd.to_datetime(train_dt['arrival_date_full'], format="%Y-%m-%d")
train_dt['status_minus_arrival_date'] = train_dt['stays_in_week_nights'] + train_dt['stays_in_weekend_nights']
y = train_dt.is_canceled
y_adr = train_dt.adr

In [212]:
train_dt = train_dt.drop(columns=['ID','is_canceled','arrival_date_year',
                                  'arrival_date_month','reservation_status',
                                  'reservation_status_date','adr','agent','company'])
train_dt['children']=train_dt['children'].fillna(train_dt['children'].median())
train_dt['country']=train_dt['country'].fillna(train_dt['country'].value_counts().index[0]) 

In [213]:
le = LabelEncoder()
categorical_features = list(train_dt.columns[train_dt.dtypes == object])
train_dt[categorical_features]=train_dt[categorical_features].apply(lambda x: le.fit_transform(x))
train_dt.head()

,hotel,lead_time,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,meal,country,market_segment,distribution_channel,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,reserved_room_type,assigned_room_type,booking_changes,deposit_type,days_in_waiting_list,customer_type,required_car_parking_spaces,total_of_special_requests,status_minus_arrival_date
0,1,342,27,1,0,0,2,0.0,0,0,124,3,1,0,0,0,2,2,3,0,0,2,0,0,0
1,0,257,27,1,0,2,1,0.0,0,2,124,5,3,0,0,0,0,0,1,0,0,2,0,0,2
2,0,257,27,1,0,2,2,0.0,0,2,124,5,3,0,0,0,0,0,0,0,0,2,0,0,2
3,0,257,27,1,0,2,2,0.0,0,2,124,5,3,0,0,0,0,0,0,0,0,2,0,0,2
4,0,257,27,1,0,2,2,0.0,0,2,124,5,3,0,0,0,0,0,0,0,0,2,0,0,2


In [214]:
ssx = StandardScaler()
print(ssx.fit(train_dt))
train_dt = ssx.transform(train_dt)
x_adr = train_dt


StandardScaler()


In [215]:
y_adr = np.array(y_adr)
y_adr = np.reshape(y_adr,(-1,1))
ssy = StandardScaler()
y_adr = ssy.fit_transform(y_adr)

In [216]:
y_adr

array([[-1.7691804 ],
       [-0.18659157],
       [-0.19643105],
       ...,
       [ 0.12002203],
       [-0.90470579],
       [-0.51447491]])

In [217]:
x_train_adr , x_val_adr, y_train_adr, y_val_adr = train_test_split(x_adr,y_adr,random_state=42)

In [218]:
x_train_adr.shape

(68648, 25)

In [223]:
rfr_params = {
    'n_estimators': [10,100,500,1000],  
    'max_features':[1,10,20],
    'max_depth': [1,5,10,20]
}

# Perform Grid Search
rfr_gs = GridSearchCV(RandomForestRegressor(random_state=49),
                 rfr_params,
                 cv = 5,
                 scoring = 'r2')
rfr = rfr_gs.fit(x_train_adr, y_train_adr.ravel())
print(f'Best Training Accuracy: {rfr.score(x_train_adr, y_train_adr)}')
print(f'Best Testing Accuracy: {rfr.score(x_val_adr, y_val_adr)}')

Best Training Accuracy: 0.9293576695889125
Best Testing Accuracy: 0.8277862881295965


In [227]:
pred_val = rfr.predict(x_val_adr)
pred_val = ssy.inverse_transform(pred_val)
print(mean_absolute_error(y_val_adr,pred_val))
print(math.sqrt(mean_squared_error(y_val_adr,pred_val)))

85.18979484977011
94.94673704761456


In [116]:
dtr_params = {
    'max_depth':[None],
    'max_features' : [20],
    'min_samples_split': [25], 
    'min_samples_leaf': [1]
}

# Perform Grid Search
dtr_gs = GridSearchCV(DecisionTreeRegressor(random_state=49),
                 dtr_params,
                 cv = 5,
                 scoring = 'r2')
dtr = dtr_gs.fit(x_train_adr, y_train_adr)
print(f'Best Training Accuracy: {dtr.score(x_train_adr, y_train_adr)}')
print(f'Best Testing Accuracy: {dtr.score(x_val_adr, y_val_adr)}')

Best Training Accuracy: 0.9002183478618
Best Testing Accuracy: 0.7488002191512195


In [118]:
svr_params = {
    'C':[5]
}

# Perform Grid Search
svr_gs = GridSearchCV(SVR(),
                 svr_params,
                 cv = 5,
                 scoring = 'r2')
svr = svr_gs.fit(x_train_adr, y_train_adr)
print(f'Best Training Accuracy: {svr.score(x_train_adr, y_train_adr)}')
print(f'Best Testing Accuracy: {svr.score(x_val_adr, y_val_adr)}')

/usr/local/lib/python3.7/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
/usr/local/lib/python3.7/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
/usr/local/lib/python3.7/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
/usr/local/lib/python3.7/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
/usr/loc

Best Training Accuracy: 0.6524571752721788
Best Testing Accuracy: 0.6989284467035404


In [120]:
lr = LinearRegression()
lr.fit(x_train_adr,y_train_adr)
print(f'Best Training Accuracy: {lr.score(x_train_adr, y_train_adr)}')
print(f'Best Testing Accuracy: {lr.score(x_val_adr, y_val_adr)}')

Best Training Accuracy: 0.3601768836142034
Best Testing Accuracy: 0.406181567130268


In [127]:
from sklearn.tree import ExtraTreeRegressor
etr_params = {
    'max_depth':[15],
    'min_samples_split': [25], 
    'min_samples_leaf': [1]
}

# Perform Grid Search
etr_gs = GridSearchCV(ExtraTreeRegressor(random_state=49),
                 etr_params,
                 cv = 5,
                 scoring = 'r2')
etr = etr_gs.fit(x_train_adr, y_train_adr)
print(f'Best Training Accuracy: {etr.score(x_train_adr, y_train_adr)}')
print(f'Best Testing Accuracy: {etr.score(x_val_adr, y_val_adr)}')

Best Training Accuracy: 0.7985532742198024
Best Testing Accuracy: 0.6963945938189189


In [128]:
from sklearn.neighbors import KNeighborsRegressor
kn = KNeighborsRegressor()
kn.fit(x_train_adr,y_train_adr)
print(f'Best Training Accuracy: {kn.score(x_train_adr, y_train_adr)}')
print(f'Best Testing Accuracy: {kn.score(x_val_adr, y_val_adr)}')

Best Training Accuracy: 0.693767970595945
Best Testing Accuracy: 0.6217321968544629


In [221]:
from sklearn.ensemble import BaggingRegressor
bag = BaggingRegressor()
bag_params = {
    'n_estimators':[1,10,100,500],
    'max_samples': [13], 
    'max_features': [1,10]
}

# Perform Grid Search
bag_gs = GridSearchCV(BaggingRegressor(random_state=49),
                 bag_params,
                 cv = 5,
                 scoring = 'r2')
bag = bag_gs.fit(x_train_adr, y_train_adr)
print(f'Best Training Accuracy: {bag.score(x_train_adr, y_train_adr)}')
print(f'Best Testing Accuracy: {bag.score(x_val_adr, y_val_adr)}')

/usr/local/lib/python3.7/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
/usr/local/lib/python3.7/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
/usr/local/lib/python3.7/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
/usr/local/lib/python3.7/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
/usr/loc

/usr/local/lib/python3.7/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
/usr/local/lib/python3.7/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
/usr/local/lib/python3.7/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
/usr/local/lib/python3.7/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
/usr/loc

Best Training Accuracy: 0.18591156941298825
Best Testing Accuracy: 0.22172419735844107


In [222]:
bagg = BaggingRegressor()
bagg.fit(x_train_adr, y_train_adr)
print(f'Best Training Accuracy: {bagg.score(x_train_adr, y_train_adr)}')
print(f'Best Testing Accuracy: {bagg.score(x_val_adr, y_val_adr)}')

/usr/local/lib/python3.7/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


Best Training Accuracy: 0.9245152080058976
Best Testing Accuracy: 0.8161896027564257


In [178]:
#train_dt = pd.read_csv('train.csv')
test_dt = pd.read_csv('test.csv')
# train_dt['arrival_date_full'] = train_dt['arrival_date_year'].astype(str) + "-" + train_dt['arrival_date_month'].map({'January':1, 'February':2, 'March':3, 'April':4, 'May':5, 'June':6, 'July':7, 'August':8, 'September':9, 'October':10, 'November':11, 'December':12}).astype(str) + "-" + train_dt['arrival_date_day_of_month'].astype(str)
# train_dt['arrival_date_full'] = pd.to_datetime(train_dt['arrival_date_full'], format="%Y-%m-%d")
test_dt['status_minus_arrival_date'] = test_dt['stays_in_week_nights'] + test_dt['stays_in_weekend_nights']
#y = train_dt.is_canceled
#y_adr = train_dt.adr



In [179]:
test_dt = test_dt.drop(columns=['ID','arrival_date_year',
                                  'arrival_date_month','agent','company'])
#train_dt['children']=train_dt['children'].fillna(train_dt['children'].median())
#test_dt['country']=train_dt['country'].fillna(train_dt['country'].value_counts().index[0])
test_dt['country']

0        DEU
1        BRA
2        PRT
3        BEL
4        FRA
        ... 
27854    GBR
27855    ITA
27856    ESP
27857    GBR
27858    PRT
Name: country, Length: 27859, dtype: object

In [180]:
le = LabelEncoder()
categorical_features = list(test_dt.columns[test_dt.dtypes == object])
test_dt[categorical_features] = test_dt[categorical_features].astype(str)
test_dt[categorical_features]=test_dt[categorical_features].apply(lambda x: le.fit_transform(x))
test_dt.shape


(27859, 25)

In [186]:
test_dt.columns == train_dt.columns 

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True])

In [185]:
train_dt.

(91531, 25)

In [187]:
ssx = StandardScaler()
test_dt = ssx.fit_transform(test_dt)
test_dt.shape

(27859, 25)

In [188]:
adr_bgg = bgg.predict(test_dt)
adr_bgg

array([ 0.28590066, -0.01803386, -0.20504586, ...,  0.00439068,
       -0.25672937,  0.04764113])

In [200]:
adr_bgg1 = ssy.inverse_transform(adr_bgg)
adr_bgg1.mean()

89.87019047595828

In [192]:
def save_reult(fin_adr, fin_can,filename):
    ans = pd.read_csv('test_nolabel.csv')
    df = pd.read_csv('test.csv')
    df['arrival_date_full'] = df['arrival_date_year'].astype(str) + "-" + df['arrival_date_month'].map({'January':1, 'February':2, 'March':3, 'April':4, 'May':5, 'June':6, 'July':7, 'August':8, 'September':9, 'October':10, 'November':11, 'December':12}).astype(str) + "-" + df['arrival_date_day_of_month'].astype(str)
    df['arrival_date_full'] = pd.to_datetime(df['arrival_date_full'], format="%Y-%m-%d")
    df['status_minus_arrival_date'] = df['stays_in_week_nights'] + df['stays_in_weekend_nights']
    tem = pd.DataFrame(fin_adr,columns=['adr'])
    tem['canceled'] = fin_can
    tem['canceled'] = np.where(tem['canceled']==0,1,0)
    tem['arrival_date_full'] = df['arrival_date_full']
    tem['status_minus_arrival_date'] = df['status_minus_arrival_date']
    tem['score'] = (tem['status_minus_arrival_date'] * tem['canceled'] * tem['adr'])
    tem = tem.groupby(tem.arrival_date_full).sum().astype(int)
    tem['score'] = ((tem['score'])/10000).astype(int)
    ans['label'] = tem['score'].values
    ans.to_csv(filename,index=False)

In [193]:
save_reult(adr_bgg1,can_pred,'fin_ans_bgg')

In [201]:
adr_rfr = rfr.predict(test_dt)
adr_rfr = ssy.inverse_transform(adr_rfr)
adr_rfr.mean()

80.60331695134492

In [197]:
save_reult(adr_rfr,can_pred,'fin_ans_rfr')

In [202]:
adr_dtr = dtr.predict(test_dt)
adr_dtr = ssy.inverse_transform(adr_dtr)
adr_dtr.mean()

79.54935874811001

In [198]:
save_reult(adr_dtr,can_pred,'fin_ans_dtr')

In [203]:
svr_adr = svr.predict(test_dt)
svr_adr = ssy.inverse_transform(svr_adr)
svr_adr.mean()

84.25832327368936

In [204]:
save_reult(svr_adr,can_pred,'fin_ans_svr')